In [1]:
import numpy as np
import re

all strings in dataset must be converted to words with normalisation

In [2]:
def get_words_list(string):
    words = string.split(' ')
    result = list()
    for word in words:
        if word == '':
            continue
        if word.find('@') != -1:
            continue
        if word.find('\n') != -1:
            continue
        if word.count('.') > 1:
            continue
        if word.count(':') > 1:
            continue
        word = re.sub(r"[#%!+-/.;,()':{}\"]*",'',word)
        if len(word) < 3:
            continue
        if word.find('[') != -1:
            word = word[1:]
        if word.find(']') != -1:
            word = word[:-1]
        if word.isdigit():
            continue

        word = word.lower()
        result.append(word)
    return result

when dictionary be created it is needed to select only unique words to make vector matrix, next function return dictionary with unique words from list of strings

In [3]:
def create_dictionary(strings):
    words = []
    for idx in range(len(strings)):
        varList = strings[idx][1]
        words += varList
    words = set(words)
    numeration = range(len(words))
    dictionary = dict(zip(words, numeration))
    return dictionary, words    

In [4]:
data = np.load('spam_messages.npz')

In [5]:
ds = list(data['data'])
ds = [a.tolist() for a in ds]

In [6]:
for idx in range(len(ds)):
    ds[idx][1] = get_words_list(ds[idx][1])

In [7]:
dictionary, words = create_dictionary(ds)

In [8]:
matrix = np.zeros((len(ds),len(dictionary)))

make matrix of vectors

In [9]:
for i in range(0,matrix.shape[0]):
    varString = ds[i][1]
    for j in range(len(varString)):
        num = dictionary.get(varString[j])
        matrix[i][num] += 1      

In [10]:
ds_x = matrix
ds_y = np.asarray(ds)[:,0].astype('int')

scale vactors with maxabsscaler

In [11]:
from sklearn.preprocessing import MaxAbsScaler

In [12]:
mas = MaxAbsScaler()
ds_x = mas.fit_transform(ds_x)

In [13]:
from sklearn.model_selection import train_test_split

split full dataset to train and test set's

In [14]:
prepare_x, test_x, prepare_y, test_y = train_test_split(ds_x, ds_y, test_size = 0.2, random_state = 42)

split train set t otrain set and validation set to adjust parameters

In [15]:
train_x, val_x, train_y, val_y = train_test_split(prepare_x, prepare_y, test_size = 0.2, random_state = 42)

load sklearn classifiers to compare ended preformance

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score

In [17]:
sgd = SGDClassifier()
knn = KNeighborsClassifier()
rfc = RandomForestClassifier()
svc = SVC()
lda = LinearDiscriminantAnalysis()
mlp = MLPClassifier()
dtf = DecisionTreeClassifier()
gnb = GaussianNB()
gpc = GaussianProcessClassifier(max_iter_predict = 2, warm_start = True)
qda = QuadraticDiscriminantAnalysis()

In [18]:
estimators = [sgd, knn, rfc, svc, lda, mlp, dtf, gnb, gpc, qda]
results = []
for estimator in estimators:
    name = str(estimator).split(sep = '(')[0]
    score = np.mean(cross_val_score(estimator, train_x, train_y, cv=10, n_jobs = -1))
    results.append(name + ' ' + str(score))
results

['SGDClassifier 1.0',
 'KNeighborsClassifier 1.0',
 'RandomForestClassifier 1.0',
 'SVC 1.0',
 'LinearDiscriminantAnalysis 1.0',
 'MLPClassifier 1.0',
 'DecisionTreeClassifier 1.0',
 'GaussianNB 1.0',
 'GaussianProcessClassifier nan',
 'QuadraticDiscriminantAnalysis 1.0']

all classifiers look's good with the exception of gpc, try mlp

In [19]:
mlp.fit(train_x, train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

measure accuracy with f1 score

In [20]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [21]:
predictions = mlp.predict(val_x)
print(f1_score(val_y, predictions))
print(precision_score(val_y, predictions))
print(recall_score(val_y, predictions))

1.0
1.0
1.0


stack train and validation set for train mlp classifier

In [22]:
train_x_full = np.vstack((train_x, val_x))
train_y_full = np.hstack((train_y, val_y))

In [23]:
mlp = MLPClassifier()
mlp.fit(train_x_full, train_y_full)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [24]:
predictions = mlp.predict(test_x)
f1_score(test_y, predictions)

1.0